## **Function Calling LLMs - Team Project**

In [30]:
import openai
import json
import os

from dotenv import load_dotenv


load_dotenv()
openai.api_key = os.environ.get("API_KEY")

## Data

In [ ]:
# prepare datasets here 
# data should be used with functions

## Functions

In [74]:
# Helper Functions

from functools import wraps
from pprint import pprint
from registry import Registry

def to_json(fcn):
    @wraps(fcn)
    def wrapper(*args, **kwargs):
        result = fcn(*args, **kwargs)
        return json.dumps(result)
    return wrapper

register = Registry()

#### API

In [75]:
@register.register
@to_json
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": 72,
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return weather_info

pprint(register.registered_functions) # all active functions

{'get_current_weather': <function get_current_weather at 0x000002202E5AB2E0>}


In [26]:
FUNCTIONS = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {
                    "type": "string", 
                    "enum": ["celsius", "fahrenheit"] # not yet captured by generate_function_metadata
                },
            },
            "required": ["location"],
        },
    }
]

# generate automatically FUNCTIONS metadata...
# from function_metadata import generate_function_metadata 

## LLM

#### Settings

In [27]:
from enum import Enum
from typing import Union

class Role(Enum):
    ASSISTANT = "assistant"
    FUNCTION = "function"
    SYSTEM = "system"
    USER = "user"

class Model_Version(Enum):
    GPT3 = "gpt-3.5-turbo-0613"
    GPT4 = "gpt-4-0613"

CONTEXT = {"role": Role.SYSTEM.value, "content": "Answer in one very short sentence."}

MAX_ITER = 5

MODEL_VERSION = Model_Version.GPT3.value

In [76]:
def _add_message_to_context(role:Role, content:str, messages:list, function_call:dict=None) -> None:
    """Add a new message to messages. This extends the LLM context."""
    if role == Role.ASSISTANT:
        d = {"role": Role.ASSISTANT.value, "content": content}
        if function_call is not None: d |= {"function_call": function_call}
        messages.append(d)

    if role == Role.USER:
        messages.append({"role": Role.USER.value, "content": content})
    
    if role == Role.FUNCTION:
        messages.append({"role": Role.FUNCTION.value, "name": function_call["name"], "content": content})
        

def send_message(role:Union[Role.USER, Role.FUNCTION], content:str, messages:list, function_call:dict=None) -> dict:
    """Send and receive a message to the LLM. Add request and response message to messages."""
    _add_message_to_context(role, content, messages, function_call)
    
    response = openai.ChatCompletion.create(
        model=MODEL_VERSION,
        messages=messages,
        function_call="auto",
        functions=FUNCTIONS,
    )
    
    response = response["choices"][0]["message"]
    response_message = response["content"]
    
    is_function_call = response.get("function_call")
    if is_function_call:
        _add_message_to_context(Role.ASSISTANT, response_message, messages, is_function_call.to_dict())
    
    return response


def handle_function(function:dict) -> json:
    """Invoke function and return result"""
    function_name, function_args = function["name"], json.loads(function["arguments"])
    
    if function_name in register.registered_functions:
        function_to_call = register[function_name]
        result = function_to_call(**function_args)
        return result


def run(user_question:str, verbose=False):    
    """Ask the LLM questions and let it run functions!"""
    messages = [CONTEXT]
    
    # Step 1: send the conversation (context) and available functions to GPT
    response = send_message(Role.USER, user_question, messages)
    if verbose: print(response)
    
    current_iteration = 1
    while (current_iteration <= MAX_ITER):
        # Step 2: check if GPT wanted to call a function 
        function_call = response.get("function_call")
        if not function_call: break
        if verbose: print(function_call)
        
        # Step 3: call the function
        function_result = handle_function(function_call)
        
        if verbose: print(function_result)
        
        # Step 4: send the info on the function call and function response to GPT
        response = send_message(Role.FUNCTION, function_result, messages, function_call)
        
        if verbose: print(response)
        
    if verbose: print(messages)
    
    return response["content"]

##### Run the LLM

In [73]:
response = run("What is 10+100?", verbose=False)
pprint(response)

<--- ANSWER --->
'110'


## Benchmark

In [58]:
obj = outer[0]

{'name': 'get_current_weather',
 'arguments': '{\n  "location": "San Francisco, CA"\n}'}